In [1]:
import numpy as np
import procrustes_module as pr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm, preprocessing
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load

In [2]:
def takeFloatsAndNames(file):
	names = []
	
	with open(file, 'r') as f:
		data = f.read().split()
		floats = []
		for elem in data:
			try:
				floats.append(float(elem))
			except ValueError:
				pass
	
	with open(file, 'r') as f:
		for line in f:
			if '.jpg' in line or '.bmp' in line or '.JPG' in line:
				if '.JPG' in line:
					line = line.replace('.JPG','.jpg')
				names.append(line.replace('\n',''))
				
	return names, floats

def compare(y_test, y_pred):
    counter = 0
    for test, pred in zip(y_test, y_pred):
        if test == pred:
            counter += 1

    print(counter/len(y_pred))

def ajust_reference(X, reference):
	X = np.reshape(X, (-1, 38))
	reference = np.reshape(reference, (-1, 38))
	
	result = []
	for x in X:
		number_list = []
		for index, ref in enumerate(reference):
			number_list.append(x[index] - ref)
		result.append(number_list)
	return result

In [3]:
names0, floats0 = takeFloatsAndNames('/home/walter/Documents/Projeto_Asas/acores_full_crop.txt')

names1, floats1 = takeFloatsAndNames('/home/walter/Documents/Projeto_Asas/peninsula_full_crop.txt')

names2, floats2 = takeFloatsAndNames('/home/walter/Documents/ruttner/ruttner_full_crop.txt')

In [4]:
y = [0 for _ in range(int(len(floats0)/38))] + [1 for _ in range(int(len(floats1)/38))] + [2 for _ in range(int(len(floats2)/38))]

floats = floats0 + floats1 + floats2
floats = np.reshape(floats,(-1,19,2))

In [5]:
X, reference = pr.procrustes_analysis(floats, 0.01)

X = ajust_reference(X, reference)
X = np.reshape(X, (-1, 38))
y = np.reshape(y, (-1))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

[[ 70. 220.]
 [ 67. 183.]
 [117. 170.]
 [202. 115.]
 [242.  67.]
 [287. 273.]
 [300. 233.]
 [297. 178.]
 [337. 131.]
 [310. 114.]
 [353.  70.]
 [340.  41.]
 [406.  74.]
 [502. 132.]
 [503.  69.]
 [505. 233.]
 [562. 158.]
 [590. 151.]
 [741.  48.]]
not quite:1851.1960620205252


In [29]:
#SVM
classifier = svm.LinearSVC(C = 500)
y_pred = classifier.fit(X_train, y_train).predict(X_test)

compare(y_test, y_pred)

0.5624161073825503


/home/walter/anaconda3/envs/w/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [7]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
y_pred = clf.fit(X_train, y_train).predict(X_test)

compare(y_test, y_pred)

0.3906040268456376


In [13]:
clf = RandomForestClassifier(n_estimators=100, max_depth=1, random_state=0)
y_pred = clf.fit(X_train, y_train).predict(X_test)

compare(y_test, y_pred)

0.5906040268456376
